# Clonamos el repositorio para obtener los dataSet

In [1]:
!git clone https://github.com/joanby/deeplearning-az.git

Cloning into 'deeplearning-az'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 10167 (delta 35), reused 44 (delta 20), pack-reused 10096
Receiving objects: 100% (10167/10167), 236.96 MiB | 24.11 MiB/s, done.
Resolving deltas: 100% (60/60), done.
Checking out files: 100% (10110/10110), done.


# Damos acceso a nuestro Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Test it

In [ ]:
!ls '/content/drive/My Drive' 

# Google colab tools

In [9]:
from google.colab import files # Para manejar los archivos y, por ejemplo, exportar a su navegador
import glob # Para manejar los archivos y, por ejemplo, exportar a su navegador
from google.colab import drive # Montar tu Google drive

# Instalar dependendias

In [10]:
!pip install sklearn

# Instalar Theano

In [11]:
!pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

  Cloning git://github.com/Theano/Theano.git to /tmp/pip-req-build-7q89qzfc
  Running command git clone -q git://github.com/Theano/Theano.git /tmp/pip-req-build-7q89qzfc
  Created wheel for Theano: filename=Theano-1.0.5+1.geb6a4125c-cp36-none-any.whl size=2668281 sha256=4d4c9648f72d9a1cfd6b33dea50b7685d703ab52da5c3aa3fe6ecd7dcad06048
  Stored in directory: /tmp/pip-ephem-wheel-cache-_6l9lz2i/wheels/ae/32/7c/62beb8371953eb20c271b3bac7d0e56e1a2020d46994346b52
Successfully built Theano
  Found existing installation: Theano 1.0.5
    Uninstalling Theano-1.0.5:
      Successfully uninstalled Theano-1.0.5


# Instalar Tensorflow y Keras


In [12]:
!pip install keras
!pip install tensorflow

# Redes Neuronales Artificales

# Parte 0 - Cómo importar las librerías y el dataset

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importar el data set


In [6]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

# Parte 1 - Pre procesado de datos

## Codificar datos categóricos

In [7]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(categories='auto'), [1])],   
    remainder='passthrough'                        
)
X = onehotencoder.fit_transform(X)
X = X[:, 1:]

## Dividir el data set en conjunto de entrenamiento y conjunto de testing

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Escalado de variables

In [9]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

# Parte 2 - Construir la RNA

## Importar Keras y librerías adicionales

In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense

## Inicializar la RNA

In [7]:
classifier = Sequential()

## Añadir las capas de entrada y primera capa oculta


In [8]:
classifier.add(Dense(units = 6, kernel_initializer = "uniform",  
                     activation = "relu", input_dim = 11))

## Añadir la segunda capa oculta

In [9]:
classifier.add(Dense(units = 6, kernel_initializer = "uniform",  activation = "relu"))

## Añadir la capa de salida

In [10]:
classifier.add(Dense(units = 1, kernel_initializer = "uniform",  activation = "sigmoid"))

## Compilar la RNA

In [11]:
classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

## Ajustamos la RNA al Conjunto de Entrenamiento

In [12]:
classifier.fit(X_train, y_train,  batch_size = 10, epochs = 100)

Epoch 1/100
800/800 [==============================] - 2s 1ms/step - loss: 0.4791 - accuracy: 0.7960
Epoch 2/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4283 - accuracy: 0.7960
Epoch 3/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4228 - accuracy: 0.8004
Epoch 4/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4185 - accuracy: 0.8230
Epoch 5/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4163 - accuracy: 0.8269
Epoch 6/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4142 - accuracy: 0.8285
Epoch 7/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4129 - accuracy: 0.8313
Epoch 8/100
800/800 [==============================] - 1s 2ms/step - loss: 0.4114 - accuracy: 0.8319
Epoch 9/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4102 - accuracy: 0.8324
Epoch 10/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4093 - accura

# Parte 3 - Evaluar el modelo y calcular predicciones finales

## Elaboramos una predicción para un cliente particular con la ANN entrenada

In [13]:
# Ver en dataset que valor corresponde a cada columna, y si es necesario o no (si eliminamos su ficticia ponemos 0,0 como en el caso de Francia)
dataset # vemos que el primer dato es francés y mujer
dataset.iloc[0,:].values
X[0,:] # vemos que el francés posee 2 ceros en las primeras 2 columnas (correspondientes a 0 en la columna de Alemania y de España, ya que eliminamos justo la ficticia de Francia)
# también vemos del mismo que posee un 0 en la columna de género, por lo que 0 es para mujer y 1 para hombre

array([0.0, 0.0, 619, 0, 42, 2, 0.0, 1, 1, 1, 101348.88], dtype=object)

In [14]:
#Utiliza nuestro modelo de RNA para predecir si el cliente con la siguiente información abandonará el banco:

#Geografia: Francia
#Puntaje de crédito: 600
#Género masculino
#Edad: 40 años de edad
#Tenencia: 3 años.
#Saldo: $ 60000
#Número de productos: 2
#¿Este cliente tiene una tarjeta de crédito? Sí
#¿Es este cliente un miembro activo? Sí
#Salario estimado: $ 50000
#Entonces, ¿deberíamos decir adiós a ese cliente?

new_prediction = classifier.predict(sc_X.transform(np.array([[0,0,600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))) # acordarse que convertimos Geografía y Género. Introducimos los datos con la transformación/escalado de sc_X, que me transforma las entradas en los datos escalados que espera la red
print(new_prediction)
print(new_prediction > 0.5) # clasifica según el umbral del 50% en True (abandona) o False (no abandona)

# El resultado es False, por lo que, con el 7% de probabilidad el cliente se irá del banco, por lo que no le tendremos que decir adios si nuestro criterio es despedir a quien supere el 50%

1/1 [==============================] - 0s 133ms/step
[[0.09149386]]
[[False]]


## Predicción de los resultados con el Conjunto de Testing

In [15]:
y_pred  = classifier.predict(X_test)
y_pred = (y_pred>0.5)

63/63 [==============================] - 0s 1ms/step


## Elaborar una matriz de confusión

In [16]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm) 
print((cm[0][0]+cm[1][1])/cm.sum()) # Accuracy de la NN con respecto a los datos de test (no se produce Overffiting, ya que es similar la precisión sobre el conjunto de train y sobre el de test. Si fuera mucho mayor en train que en test si)
# Forma de evaluar las predicciones van a ser las mismas que en los algoritmos de ML
# Comparar resultado de predicción sobre test con el conseguido sobre train en el entrenamiento de la red para ver si hay Overffiting

[[1545   50]
 [ 260  145]]
0.845


# Parte 4 - Evaluar, mejorar y Ajustar la RNA


## Evaluar la **RNA**


In [17]:
#!pip uninstall keras -y
#!pip install keras==2.12.0
#!pip install tensorflow==2.12.0

In [18]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score # validación cruzada

In [19]:
# Definimos una función que cree la red neuronal con KerasClassifier. El reescalado me hará falta en el contexto del k-fold, no en esta etapa.
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout # Dropout, se aplica para evitar sobreajuste luego de cada capa que requiera aprender (en nuestro problema no es necesario, ya que hay poca varianza en las iteraciones del k-fold)

def build_classifier():
  classifier = Sequential()
  classifier.add(Dense(units = 6, kernel_initializer = "uniform", activation = "relu", input_dim = 11))
  #classifier.add(Dropout(p = 0.1)) # p: probabilidad de ser desactivadas las neuronas (si tenemos 10 y p=0.1, 1 de las 10 neuronas será desactivada por iteración), aumentar p cuando veamos que no se resuelve el Overffiting (sin pasarnos de 0.5 para no entrar en Underffiting)
  classifier.add(Dense(units = 6, kernel_initializer = "uniform", activation = "relu"))
  #classifier.add(Dropout(p = 0.1))
  classifier.add(Dense(units = 1, kernel_initializer = "uniform", activation = "sigmoid"))
  classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
  return classifier

In [31]:
# Creamos el clasificador con Keras (hacer de cuenta que el código empieza en la parte 4)
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, nb_epoch = 100) # clasificador de keras. build_fn: función de generación de arquitectura de red, batch_size y nb_epoch: tamaño del bloque que una vez finalizado su procesado se actualizan pesos y el número de épocas (igual que antes)

# En nuestro caso, vamos a aplicar antes la división en train/test (opcional) y el reescalado de las variables (X_train y Y_train si optamos por div por % antes o sobre X y Y si no)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

# Aplicamos validación cruzada (k-fold) a la ANN en vez de división en train/test almacenando los resultados en un vector de precisiones llamado 'accuracies'
accuracies = cross_val_score(estimator=classifier, X = X_train, y = y_train, cv = 10, n_jobs=-1, verbose = 1) # estimator: objeto para ajustar los datos, X y Y: las VI y VD de para aplicar CV, cv: número de folds a utilizar (generalmente se usa 10, siendo un buen criterio de uso en términos de eficacia/rapidez), si no nos da buenos resultados podemos optar por un número mayor de folds, n_jobs: -1 para utilizar todos los procesadores de las máquinas en paralelo (usamos toda la potencia del ordenador), 
# El entrenamiento se realizará tomando como train los 9 folds de la iteración, y el restante como test, sobre el X y Y aclarado
# En vez de hacerlo con X_train y Y_train lo podemos hacer con todo el set de datos directamente, poniendo X y Y, pero con esta opción nos guardamos una parte de las variables para luego ver como el modelo reacciona ante otros datos no usados para el entrenamiento con kfolds (a pesar de que cada resultado de fold es válido), siendo necesario de aplicar en este caso la división en train/test antes

C:\Users\Maxi Mores\AppData\Local\Temp\ipykernel_4032\702602064.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, nb_epoch = 100) # clasificador de keras. build_fn: función de generación de arquitectura de red, batch_size y nb_epoch: tamaño del bloque que una vez finalizado su procesado se actualizan pesos y el número de épocas (igual que antes)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    4.2s remaining:    2.8s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    6.5s finished


In [32]:
accuracies # vemos las precisiones obtenidas en cada iteración
# Rendimientos inferiores a las obtenidas anteriormente solo con train/test, pero con poca varianza (indicio de que la partición específica usada en train/test estaba por encima del rendimiento promedio del modelo)

array([0.78625   , 0.79000002, 0.80000001, 0.78250003, 0.81625003,
       0.81      , 0.78750002, 0.79374999, 0.79874998, 0.79500002])

In [33]:
mean = accuracies.mean() 
variance = accuracies.std()
print(f'Media de rendimiento sobre el set de train: {mean}')
print(f'Desviación std de las 10 iteraciones sobre el set de train: {variance}')
# La precisión media es el 80% aprox, con una varianza del 1,2%. A lo mejor podríamos exigirle a la ANN un poco más de precisión, pero no varianza
# Intentemos buscar una red que llegue al 85-86 % de rendimiento, ya que esta media nos indica la precisión en el entrenamiento validado sobre datos no vistos durante la iteración, osea que es un rendimiento válido (se hace sobre datos no usados para entrenamiento, solo que en vez de usar todo el dataset usamos solo la parte de train)

Media de rendimiento sobre el set entero: 0.7960000097751617
Desviación std de las 10 iteraciones sobre el set entero: 0.010105690527913859


## Mejorar la RNA

#### Regularización de Dropout para evitar el *overfitting*


In [ ]:
# El sobreajuste es cuando nuestro modelo aprende demasiado bien sobre los datos de entrenamiento, faltandolé efectividad al momento de enfrentarse con datos nuevos (test)
# Podriamos identificar un caso de Overffiting en una ANN notando una gran varianza en las iteraciones del k-fold validation, ya que las correlaciones que el modelo aprende muy bien de los folds de entrenamiento no son aplicables al fold de test de similar manera en cada iteración, dependiendo más así de la división específica de los folds
# Si el desempeño de cada iteración es similar, es síntoma de que el rendimiento del modelo no depende tanto de la asignación de datos en folds, siendo capaz de captar patrones/tendencias aplicables sobre datos no usados para entrenar

In [ ]:
# Si tenemos mucha varianza en las iteraciones debemos corregirla con la capa Dropout. Esta capa descativa aleatoriamente neuronas de la capa anterior en cada iteración, para que las mismas no se especialicen sobre los mismos rasgos (se sobreescriban) y no sean demasiado dependientes
# Esto nos garantiza que las correlaciones que aprenda cada neurona serán más independientes una de la otra 
# Dropout se puede aplicar luego de cualquier capa con neuronas que deban aprender. Conviene aplciarlo después de todas las capas, e ir incrementando la cantidad de capas Dropout y la probabilidad de desactivarse a medida de que veamos que no soluciona el Overffiting
# También conviene tener muchas capas de Dropout con bajo valor p despues de cada capa densa, en vez de tener solo una con un valor p alto (fuerte prob de olvido)

## Ajustar la *RNA*

In [ ]:
# Buscaremos mejorar el rendimiento de la red con Grid Search, buscando los hiperparámetros óptimos de la arquitectura para encontrar el mejor rendimiento
# Grid Search nos devolverá la mejor combinación de hiperparámetros para conseguir el rendimiento más elevado

In [34]:
from sklearn.model_selection import GridSearchCV

In [35]:
# Volvemos a definir la arquitectura de la ANN, no agregamos el optimizador, el cual vamos a incluir en la búsqueda de hiperparámetros óptimos
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

def build_classifier(optimizer):
  classifier = Sequential()
  classifier.add(Dense(units = 6, kernel_initializer = "uniform",  activation = "relu", input_dim = 11))
  classifier.add(Dense(units = 6, kernel_initializer = "uniform",  activation = "relu"))
  classifier.add(Dense(units = 1, kernel_initializer = "uniform",  activation = "sigmoid"))
  classifier.compile(optimizer = optimizer, loss = "binary_crossentropy", metrics = ["accuracy"])  
  return classifier

# Si queremos buscar un hiperparámetro presente en la arquitectura, se lo otorgamos a la función como parámetro y ponemos la variable en donde se pide el hiperparámetro. Podemos agregar varios parámetros, separándolos por comas en los argumentos

In [36]:
# Ahora al classifier no le aclaramos ni el batch_size ni el número de épocas, ya que esos serán algunos de los hiperparámetros que buscaremos el óptimo
classifier = KerasClassifier(build_fn = build_classifier)

C:\Users\Maxi Mores\AppData\Local\Temp\ipykernel_4032\1443411995.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn = build_classifier)


In [40]:
# Hacemos una lista de los hiperparámetros que queremos optimizar seguida de una lista de los valores que queremos que pruebe (ojo con poner muchas combinaciones)
parameters = {
    'batch_size' : [25,32], # mientras mayor es el bloque, más datos pasan por la ANN antes de actualizar los pesos
    'nb_epoch' : [100, 500], 
    'optimizer' : ['adam', 'rmsprop']
}
# Podemos optimizar:
# - el tamaño del bloque (batch_size) ---> probar con potencias de 2
# - el número de épocas (nb_epoch) ---> probar más épocas a ver si llegamos a convergencia
# - el número de neuronas de las capas (units)
# - el tipo de kernel (kernel_initializer) 
# - la función de activación (activation) 
# - la métrica (no tan relevante)
# - el optimizador (optimizer) ---> rmsprop esta basado en el gradiente descendiente y es recomendado para ANN regulares (mientras que adam es el recomendado para ANN más complejas)

In [41]:
# Usamos GridSearchCV para combinar Cross Validation y la búsqueda de hiperparámetros configurada en parameters
grid_search = GridSearchCV(estimator = classifier, 
                           param_grid = parameters, 
                           scoring = 'accuracy', # indicamos la medida con la que queremos comparar los resultados de combinación de parámetros (ver que hay más parámetros)
                           cv = 10)

grid_search = grid_search.fit(X_train, y_train) # usamos el objeto para que se ajuste a los datos de entrenamiento

# (investigar más propiedades/atributos del objeto de grid_search)
best_parameters = grid_search.best_params_ # mejor selección de parámetros
best_accuracy = grid_search.best_score_ # resultado obtenido (según la métrica especificada) con la mejor combinación de hiperparámetros

320/320 [==============================] - 1s 1ms/step - loss: 0.5623 - accuracy: 0.7960


In [42]:
print(f'Mejores parámetros: {best_parameters}')
print(f'Mejor accuracy (métrica): {best_accuracy}')
# Vemos que, como fueron pocas la combinaciones de hiperparámetros que pedimos y no fueron las mejores opciones a buscar, pero podemos y debemos seguir buscando

Mejores parámetros: {'batch_size': 25, 'nb_epoch': 100, 'optimizer': 'rmsprop'}
Mejor accuracy (métrica): 0.796


# Parte 5 - Obtener mejores resultados con Grid Search

In [1]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score # validación cruzada

In [2]:
# Volvemos a definir la arquitectura de la ANN, no agregamos el optimizador, el cual vamos a incluir en la búsqueda de hiperparámetros óptimos
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

def build_classifier(optimizer):
  classifier = Sequential()
  classifier.add(Dense(units = 22, kernel_initializer = "uniform",  activation = "relu", input_dim = 11))
  classifier.add(Dense(units = 22, kernel_initializer = "uniform",  activation = "relu"))
  classifier.add(Dense(units = 11, kernel_initializer = "uniform",  activation = "relu")) # tercera capa oculta
  classifier.add(Dense(units = 1, kernel_initializer = "uniform",  activation = "sigmoid"))
  classifier.compile(optimizer = optimizer, loss = "binary_crossentropy", metrics = ["accuracy"])  
  return classifier

# Si queremos buscar un hiperparámetro presente en la arquitectura, se lo otorgamos a la función como parámetro y ponemos la variable en donde se pide el hiperparámetro. Podemos agregar varios parámetros, separándolos por comas en los argumentos

In [3]:
# Ahora al classifier no le aclaramos ni el batch_size ni el número de épocas, ya que esos serán algunos de los hiperparámetros que buscaremos el óptimo
classifier = KerasClassifier(build_fn = build_classifier)

C:\Users\Maxi Mores\AppData\Local\Temp\ipykernel_14136\1443411995.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn = build_classifier)


In [4]:
# Hacemos una lista de los hiperparámetros que queremos optimizar seguida de una lista de los valores que queremos que pruebe (ojo con poner muchas combinaciones)
parameters = {
    'batch_size' : [10, 20], # mientras mayor es el bloque, más datos pasan por la ANN antes de actualizar los pesos
    'nb_epoch' : [ 20, 30, 50], 
    'optimizer' : ['adam', 'rmsprop'],
    'validation_split': [0.1] # porcentaje de los datos que guardaremos para validar el entrenamiento y evitar el overffiting
}
# multiplicar opciones para ver si el número de combinaciones no es muy elevado

# Podemos optimizar:
# - el tamaño del bloque (batch_size) ---> probar con potencias de 2
# - el número de épocas (nb_epoch) ---> probar más épocas a ver si llegamos a convergencia
# - el número de neuronas de las capas (units)
# - el tipo de kernel (kernel_initializer) 
# - la función de activación (activation) 
# - la métrica (no tan relevante)
# - el optimizador (optimizer) ---> rmsprop esta basado en el gradiente descendiente y es recomendado para ANN regulares (mientras que adam es el recomendado para ANN más complejas)

In [9]:
# Ejecutar la parte 0 y parte 1 (Obtención de Datos y Preprocesado de Datos)

In [10]:
# Usamos GridSearchCV para combinar Cross Validation y la búsqueda de hiperparámetros configurada en parameters
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(estimator = classifier, 
                           param_grid = parameters, 
                           scoring = 'accuracy', # indicamos la medida con la que queremos comparar los resultados de combinación de parámetros (ver que hay más parámetros)
                           cv = 10,
                           n_jobs = -1)

grid_search = grid_search.fit(X_train, y_train) # usamos el objeto para que se ajuste a los datos de entrenamiento

# (investigar más propiedades/atributos del objeto de grid_search)
best_parameters = grid_search.best_params_ # mejor selección de parámetros
best_accuracy = grid_search.best_score_ # resultado obtenido (según la métrica especificada) con la mejor combinación de hiperparámetros

720/720 [==============================] - 2s 2ms/step - loss: 0.4661 - accuracy: 0.7954 - val_loss: 0.4156 - val_accuracy: 0.7950


In [11]:
print(f'Mejores parámetros: {best_parameters}')
print(f'Mejor accuracy (métrica): {best_accuracy}')

Mejores parámetros: {'batch_size': 10, 'nb_epoch': 50, 'optimizer': 'adam', 'validation_split': 0.1}
Mejor accuracy (métrica): 0.797625
